In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset,random_split
import random
import os

In [2]:
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [4]:
data1 = np.load('./data/data0.npy')
lab1 = np.load('./data/lab0.npy')
data2 = np.load('./data/data1.npy')
lab2 = np.load('./data/lab1.npy')
data3 = np.load('./data/data2.npy')
lab3 = np.load('./data/lab2.npy')

In [5]:
data_cumulative = np.concat([data1,data2,data3])
data_cumulative = data_cumulative[:, np.newaxis, :, :]
labels_cumulative = np.concat([lab1,lab2,lab3])
data = torch.tensor(data_cumulative, dtype=torch.float32).to(device)
labels = torch.tensor(labels_cumulative, dtype=torch.long).to(device)

In [6]:
dataset = TensorDataset(data, labels)

In [8]:
batch_size = 64
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [10]:
class CNNModel(nn.Module):
    def __init__(self, num_classes=37):
        super(CNNModel, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        
        self.fc1 = nn.Linear(128 * 5 * 21, 256)
        self.fc2 = nn.Linear(256, num_classes)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.7)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        x = x.view(x.size(0), -1)
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x


In [11]:
model = torch.load("model.pth").to(device)

/tmp/ipykernel_15898/1460648578.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("model.pth").to(device)


In [12]:
total_val = 0
correct_val = 0
for images, labels in loader:
    images, labels = images.to(device), labels.to(device)
    output = model(images)
    _, predicted = torch.max(output.data, 1)
    total_val += len(labels)
    correct_val += (predicted == labels).sum().item()
print(f"Accuracy is {correct_val/total_val}%")

Accuracy is 0.16086666666666666%
